In [ ]:
import torch

inputs = torch.tensor(
    [
        [0.43, 0.15, 0.89],  # Your
        [0.55, 0.87, 0.66],  # journey
        [0.57, 0.85, 0.64],  # starts
        [0.22, 0.58, 0.33],  # with
        [0.77, 0.25, 0.10],  # one
        [0.05, 0.80, 0.55],  # step
    ]
)

query = inputs[1]  # [0.55, 0.87, 0.66]
attn_scores = torch.empty(inputs.shape[0])
print(attn_scores.shape)  # 这里需要算出与query注意力
for i, k in enumerate(inputs):
    attn_scores[i] = torch.dot(k, query)
attn_weights = torch.softmax(attn_scores, dim=-1)
context_vec = torch.zeros(query.shape)
for i, k in enumerate(inputs):
    context_vec += attn_weights[i] * k
print(context_vec)

将这个逻辑平铺到到每个token上

In [ ]:
attn_scores = torch.empty(inputs.shape[0], inputs.shape[0])
attn_scores = inputs @ inputs.T
print(attn_scores.shape)
attn_weights = torch.softmax(attn_scores, dim=-1)
context_vecs = attn_weights @ inputs
print(context_vecs)

# 我们将还没推算进去的token 也算了进去，所以我们需要一个mask来过滤掉这些toke
mask = torch.triu(torch.ones(inputs.shape[0], inputs.shape[0]), diagonal=1)
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
attn_weights = torch.softmax(masked / inputs.shape[-1] ** 0.5, dim=-1)
print(attn_weights)
context_vecs = attn_weights @ inputs
print(context_vecs)

# 添加 Dropout
dropout = torch.nn.Dropout(0.5)
dropout(attn_weights)

In [ ]:
import attention

torch.manual_seed(789)
d_in = inputs.shape[1]
d_out = 2
sa = attention.SelfAttention(d_in, d_out)
print("SelfAttention output:\n", sa(inputs))

In [ ]:
torch.manual_seed(789)
batch = torch.stack((inputs, inputs), dim=0)
context_length = batch.shape[1]
csa = attention.CausalSelfAttention(d_in, d_out, context_length, 0.0)
print("CausalSelfAttention output:\n", csa(batch))

In [ ]:
torch.manual_seed(789)
batch = torch.stack((inputs, inputs), dim=0)
context_length = batch.shape[1]
mha = attention.MulitHeadAttention(d_in, d_out, context_length, 0.0, 2)
print("MulitHeadAttention output:\n", mha(batch))